<a href="https://colab.research.google.com/github/eysteinn13/bert-skh/blob/main/contentImageSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/eysteinn13/bert-skh.git

Cloning into 'bert-skh'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


Welcome to this notebook


In [ ]:
%pip install scikit-learn sentence-transformers

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import json

In [5]:
import zipfile

def get_descriptions():
    with open("bert-skh/new_desc.json", "r") as fp:
        return json.load(fp)


def get_embeddings():
    with zipfile.ZipFile("bert-skh/embeddings.json.zip", 'r') as zip_ref:
        zip_ref.extractall("bert-skh/")
        with open("bert-skh/embeddings.json", 'r') as unzipped:
            return json.load(unzipped)


def get_images():
    with open("bert-skh/images.json", "r") as fp:
        return json.load(fp)


In [6]:
def calc_similarity(new_embedding, n=10):
    embeddings = get_embeddings()
    results = {}
    for key, embedding in embeddings.items():
        results[key] = cosine_similarity([new_embedding], [embedding])[0][0]
    top_results = sorted(results, key=results.get)[::-1]
    return top_results[:n]

In [7]:
def get_most_similar_images(text_input: str, n=5):
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    input_embedding = model.encode(text_input)
    most_similar = calc_similarity(input_embedding, n)
    images = get_images()
    descriptions = get_descriptions()
    found = []
    for similar in most_similar:
        for image in images:
            description_text = ""
            if image['content_item_locale_id'] == similar:
                for desc in descriptions:
                    if desc['content_item_locale_id'] == similar:
                        description_text = desc['content_item_value']
                image_url = image['content_item_value']
                if image['content_item_id'] not in found:
                    found.append(image['content_item_id'])
                    print(f"Image: {image_url} - Description: {description_text} - ID: {image['content_item_id']} - Locale: {image['content_item_locale_id']}")

In [9]:
get_most_similar_images("Happy wife happy life", n=10)

Image: https://storage.googleapis.com/glmsidekicklive_cms/imageCardItem/12630/en/image.jpg?v=1660057336966 - Description: To many, success means achieving goals. Promotions at work, nice homes, and strong romantic relationships are all considered measures of success by society.  - ID: 12630 - Locale: 40151
Image: https://storage.googleapis.com/glmsidekicklive_cms/imageCardItem/3792/en/image.png?v=1636038320656 - Description: Consistency is key to good results. That means keeping your skin happy even when it looks clear and you feel cured. - ID: 3792 - Locale: 19643
Image: https://storage.googleapis.com/glmsidekicklive_cms/imageCardItem/6289/en/image.jpg?v=1647624407092 - Description: Gratitude means noticing positive outcomes in life. Being thankful for the things you have, tangible and intangible. Showing appreciation for kindness and returning it. - ID: 6289 - Locale: 14715
Image: https://storage.googleapis.com/glmsidekicklive_cms/imageCardItem/4583/en/image.jpg?v=1640012149907 - Des